<a href="https://colab.research.google.com/github/CodeT1m/agrik_obj_detect/blob/master/AgrikAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("Hello colab")

Hello colab


In [4]:
!pwd

/content


In [5]:
!git --version

git version 2.17.1


In [8]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/CodeT1m/agrikfyp'

# Number of training steps.
num_steps = 800000  # 200000

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

print(repo_url)

https://github.com/CodeT1m/agrikfyp


# Clone the object_detection_demo repository or fork 

In [10]:
import os

%cd /content
print("ok")
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
print(repo_dir_path)
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
ok
/content/agrikfyp
Cloning into 'agrikfyp'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: '/content/agrikfyp'
/content
fatal: not a git repository (or any of the parent directories): .git
